First, let's import the Python bindings, as usual.

In [1]:
import metapy

In [2]:
metapy.__version__ # you will want your version to be >= to this

'0.2.3'

If you would like to, you can inform MeTA to output log data to stderr like so:

In [3]:
metapy.log_to_stderr()

Now, let's download a list of stopwords and a sample dataset to begin exploring MeTA's topic models.

In [4]:
!wget -N https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt

--2017-07-31 17:19:53--  https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747 (2.7K) [text/plain]
Saving to: ‘lemur-stopwords.txt’

lemur-stopwords.txt 100%[===================>]   2.68K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2017-07-31 17:19:54 (47.6 MB/s) - ‘lemur-stopwords.txt’ saved [2747/2747]



In [5]:
!wget -N https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
!tar xf ceeaus.tar.gz

--2017-07-31 17:19:54--  https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving meta-toolkit.org... 50.116.41.177, 2600:3c02::f03c:91ff:feae:b777
Connecting to meta-toolkit.org|50.116.41.177|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ceeaus.tar.gz’ not modified on server. Omitting download.



We will need to index our data to proceed. We eventually want to be able to extract the bag-of-words representation for our individual documents, so we will want a `ForwardIndex` in this case.

In [6]:
fidx = metapy.index.make_forward_index('[ceeaus-config.toml](https://github.com/meta-toolkit/metapy/blob/master/tutorials/ceeaus-config.toml)')

Just like in classification, the feature set used for the topic modeling will be the feature set used at the time of indexing, so if you want to play with a different set of features (like bigram words), you will need to re-index your data.

For now, we've just stuck with the default filter chain for unigram words, so we're operating in the traditional bag-of-words space.

Let's load our documents into memory to run the topic model inference now.

In [7]:
dset = metapy.learn.Dataset(fidx)

Now, let's try to find some topics for this dataset. To do so, we're going to use a generative model called a topic model.

There are many different topic models in the literature, but the most commonly used topic model is Latent Dirichlet Allocation. Here, we propose that there are K topics (represented with a categorical distribution over words) $\phi_k$ from which all of our documents are genereated. These K topics are modeled as being sampled from a Dirichlet distribution with parameter $\vec{\alpha}$. Then, to generate a document $d$, we first sample a distribution over the K topics $\theta_d$ from another Dirichlet distribution with parameter $\vec{\beta}$. Then, for each word in this document, we first sample a topic identifier $z \sim \theta_d$ and then the word by drawing from the topic we selected ($w \sim \phi_z$). Refer to the [Wikipedia article on LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) for more information.

The goal of running inference for an LDA model is to infer the latent variables $\phi_k$ and $\theta_d$ for all of the $K$ topics and $D$ documents, respectively. MeTA provides a number of different inference algorithms for LDA, as each one entails a different set of trade-offs (inference in LDA is intractable, so all inference algorithms are approximations; different algorithms entail different approximation guarantees, running times, and required memroy consumption). For now, let's run a Variational Infernce algorithm called CVB0 to find two topics. (In practice you will likely be finding many more topics than just two, but this is a very small toy dataset.)

In [8]:
lda_inf = metapy.topics.LDACollapsedVB(dset, num_topics=2, alpha=1.0, beta=0.01)
lda_inf.run(num_iters=1000)

The above ran the CVB0 algorithm for 1000 iterations, or until an algorithm-specific convergence criterion was met. Now let's save the current estimate for our topics and topic proportions.

In [10]:
lda_inf.save('lda-cvb0')

We can interrogate the topic inference results by using the `TopicModel` query class. Let's load our inference results back in.

In [11]:
model = metapy.topics.TopicModel('lda-cvb0')

Now, let's have a look at our topics. A typical way of doing this is to print the top $k$ words in each topic, so let's do that.

In [13]:
model.top_k(tid=0)

[(3341, 0.13110421021038324),
 (3045, 0.05434941973171466),
 (2677, 0.03678014124195563),
 (3346, 0.03349270116563351),
 (281, 0.0225307149144554),
 (3729, 0.015620504332737565),
 (1953, 0.012780941699372448),
 (707, 0.012635092781408291),
 (592, 0.011987205216970597),
 (2448, 0.011317761088081552)]

The models operate on term ids instead of raw text strings, so let's convert this to a human readable format by using the vocabulary contained in our `ForwardIndex` to map the term ids to strings.

In [14]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0)]

[('smoke', 0.13110421021038324),
 ('restaur', 0.05434941973171466),
 ('peopl', 0.03678014124195563),
 ('smoker', 0.03349270116563351),
 ('ban', 0.0225307149144554),
 ('think', 0.015620504332737565),
 ('japan', 0.012780941699372448),
 ('complet', 0.012635092781408291),
 ('cigarett', 0.011987205216970597),
 ('non', 0.011317761088081552)]

In [15]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1)]

[('time', 0.06705633566728429),
 ('job', 0.056059202698078466),
 ('part', 0.052222972425643444),
 ('student', 0.04642930410399952),
 ('colleg', 0.03488134676327594),
 ('work', 0.029067430777979537),
 ('money', 0.02885016964119207),
 ('think', 0.022331317888389407),
 ('import', 0.02075566243543525),
 ('studi', 0.015483008722512084)]

We can pretty clearly see that this particular dataset was about two major issues: smoking in public and part time jobs for students. This dataset is actually a collection of essays written by students, and there just so happen to be two different topics they can choose from!

The topics are pretty clear in this case, but in some cases it is also useful to score the terms in a topic using some function of the probability of the word in the topic and the probability of the word in the other topics. Intuitively, we might want to select words from each topic that best reflect that topic's content by picking words that both have high probability in that topic **and** have low probability in the other topics. In other words, we want to balance between high probability terms and highly specific terms (this is kind of like a tf-idf weighting). One such scoring function is provided by the toolkit in `BLTermScorer`, which implements a scoring function proposed by Blei and Lafferty.

In [27]:
scorer = metapy.topics.BLTermScorer(model)
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0, scorer=scorer)]

[('smoke', 0.8741661352396957),
 ('restaur', 0.3174611854294489),
 ('smoker', 0.20060321654683078),
 ('ban', 0.12853060169345398),
 ('cigarett', 0.06557618816704736),
 ('non', 0.06128436226482625),
 ('complet', 0.061053888089489305),
 ('japan', 0.05846310193983406),
 ('health', 0.050548462085096446),
 ('seat', 0.04533997725235284)]

In [26]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1, scorer=scorer)]

[('job', 0.348219801380548),
 ('part', 0.3131101849492852),
 ('student', 0.283288839381727),
 ('colleg', 0.20808951747041338),
 ('time', 0.17797627851908793),
 ('money', 0.16234654268837612),
 ('work', 0.15585386268480006),
 ('studi', 0.08228275310960287),
 ('learn', 0.06491893059362819),
 ('experi', 0.05494520825302107)]

Here we can see that the uninformative word stem "think" was downweighted from the word list from each topic, since it had relatively high probability in either topic.

We can also see the inferred topic distribution for each document.

In [18]:
model.topic_distribution(0)

<metapy.stats.Multinomial {0: 0.021341, 1: 0.978659}>

It looks like our first document was written by a student who chose the part-time job essay topic...

In [23]:
model.topic_distribution(900)

<metapy.stats.Multinomial {0: 0.978797, 1: 0.021203}>

...whereas this document looks like it was written by a student who chose the public smoking essay topic.